In [1]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter, HyperparameterTuner
from sagemaker import get_execution_role
import os

print("--- ⚙️ Step 1: Initializing SageMaker Environment ---")

# 1. 获取执行角色 (IAM Role)
# 如果在本地 VS Code 运行，可能需要手动指定 Role ARN
try:
    role = get_execution_role()
except ValueError:
    print("⚠️ 警告: 无法自动获取角色，请手动填入 ARN。")
    # 请去 AWS Console -> IAM -> Roles 找一个类似 AmazonSageMaker-ExecutionRole 的角色
    role = "arn:aws:iam::137568342316:role/SageMakerExecutionRole" 

# 2. 基础配置
sagemaker_session = sagemaker.Session()
bucket_name = 'sleep-disorder-mlops-bucket' # 您的 S3 桶名

# 3. 指定代码源 (Source of Truth for Code)
# SageMaker 会自动 `git clone` 这个仓库到训练实例中
git_repo = 'https://github.com/vinnie071015/sleeping-disorder-mlops.git' # 替换为您的仓库地址
git_config = {
    'repo': git_repo, 
    'branch': 'main'
}

print(f"✅ Role: {role}")
print(f"✅ Bucket: {bucket_name}")
print(f"✅ Git Repo: {git_repo}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/zhengchengsheng/Library/Application Support/sagemaker/config.yaml
--- ⚙️ Step 1: Initializing SageMaker Environment ---


Couldn't call 'get_role' to get Role ARN from role name ML_Project to get Role path.


⚠️ 警告: 无法自动获取角色，请手动填入 ARN。
✅ Role: arn:aws:iam::137568342316:role/SageMakerExecutionRole
✅ Bucket: sleep-disorder-mlops-bucket
✅ Git Repo: https://github.com/vinnie071015/sleeping-disorder-mlops.git


In [2]:
print("\n--- 💾 Step 2: Defining S3 Data Input ---")

# 定义 S3 数据输入
# SageMaker 会自动把这个 S3 路径下的所有文件下载到容器内的 /opt/ml/input/data/train/
# 这里的 s3_data 必须指向包含 sleep_data.csv 的文件夹路径 (以 / 结尾)
s3_input_path = f's3://{bucket_name}/raw_data/'

train_input = sagemaker.inputs.TrainingInput(
    s3_data=s3_input_path, 
    content_type='text/csv'
)

print(f"✅ Training Data Source: {s3_input_path}")


--- 💾 Step 2: Defining S3 Data Input ---
✅ Training Data Source: s3://sleep-disorder-mlops-bucket/raw_data/


In [3]:
print("\n--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---")

# 使用 ml.m5.large (通用型)
instance_type = 'ml.m5.large'

common_estimator_args = {
    # 修正 1: 入口文件路径要包含 src (因为 source_dir 变成了根目录)
    'entry_point': 'src/train.py',
    
    # 修正 2: source_dir 设为 '.' (代表 Git 仓库的根目录)
    # 这样 SageMaker 能同时看到 src/ 文件夹和 requirements.txt
    'source_dir': '.',
    
    'role': role,
    'instance_count': 1,
    'instance_type': instance_type,
    'framework_version': '1.2-1',
    'py_version': 'py3',
    'git_config': git_config,
    'sagemaker_session': sagemaker_session,
    
    # 修正 3: 不需要手动指定 dependencies 了
    # 因为 requirements.txt 就在 source_dir (根目录) 下，SageMaker 会自动识别并安装
    # 'dependencies': [...] <--- 删除这行
}

print(f"✅ Instance Type: {instance_type}")
print(f"✅ Path Correction: source_dir='.', entry_point='src/train.py'")


--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---
✅ Instance Type: ml.m5.large
✅ Path Correction: source_dir='.', entry_point='src/train.py'


In [4]:
print("\n--- 🎛️ Step 4: Defining Tuners for 3 Models ---")

# 定义指标抓取规则 (对应 src/train.py 中的 print 语句)
metric_definitions = [
    {'Name': 'accuracy', 'Regex': '✅ Accuracy: ([0-9\\.]+)'},
    {'Name': 'f1', 'Regex': '✅ F1 Score: ([0-9\\.]+)'}
]

# ==========================================
# A. Random Forest Tuner
# ==========================================
rf_estimator = SKLearn(**common_estimator_args)
# 固定模型类型为 RF
rf_estimator.set_hyperparameters(model_type='random_forest')

rf_tuner = HyperparameterTuner(
    estimator=rf_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'n_estimators': IntegerParameter(50, 150),
        'max_depth': IntegerParameter(5, 15)
    },
    max_jobs=2,          # 总共跑 2 次 (省钱)
    max_parallel_jobs=1, # 串行跑 (安全)
    base_tuning_job_name='rf-tuning'
)

# ==========================================
# B. SVM Tuner
# ==========================================
svm_estimator = SKLearn(**common_estimator_args)
svm_estimator.set_hyperparameters(model_type='svm')

svm_tuner = HyperparameterTuner(
    estimator=svm_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0),
        'kernel': CategoricalParameter(['rbf', 'linear'])
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='svm-tuning'
)

# ==========================================
# C. Logistic Regression Tuner
# ==========================================
lr_estimator = SKLearn(**common_estimator_args)
lr_estimator.set_hyperparameters(model_type='logistic_regression')

lr_tuner = HyperparameterTuner(
    estimator=lr_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0)
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='lr-tuning'
)

print("✅ Tuners for RF, SVM, and LR are ready.")

/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)



--- 🎛️ Step 4: Defining Tuners for 3 Models ---
✅ Tuners for RF, SVM, and LR are ready.


In [14]:
import time

print("\n--- 🚀 Step 5: Launching Jobs Sequentially ---")

# 1. 运行 Random Forest
print(f"\n🌲 [1/3] Launching Random Forest Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
# SageMaker 会自动生成唯一的 Job Name，这里 fit 会直接打印出来
rf_tuner.fit({'train': train_input}, wait=True) 
print("✅ Random Forest Tuning Completed!")

# 2. 运行 SVM
print(f"\n🛡️ [2/3] Launching SVM Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
svm_tuner.fit({'train': train_input}, wait=True)
print("✅ SVM Tuning Completed!")

# 3. 运行 Logistic Regression
print(f"\n📈 [3/3] Launching Logistic Regression Tuning... (Timestamp: {time.strftime('%H:%M:%S')})")
lr_tuner.fit({'train': train_input}, wait=True)
print("✅ Logistic Regression Tuning Completed!")

print("\n🎉 All experiments finished!")


--- 🚀 Step 5: Launching Jobs Sequentially ---

🌲 [1/3] Launching Random Forest Tuning... (Timestamp: 10:38:31)


Cloning into '/var/folders/py/wsw9dv2n5g70rl8sm3pxxjg40000gn/T/tmphzjps5_c'...
Already on 'main'


Your branch is up to date with 'origin/main'.


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.....................................................*


UnexpectedStatusException: Error for HyperParameterTuning job rf-tuning-251203-1038: Failed. Reason: All training jobs failed. Please take a look at the training jobs failures to get more details.. Check troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html

In [16]:
import boto3
import time
import re
from botocore.exceptions import ClientError

# 确保在运行这段代码之前，您已经有一个失败的调优任务名称
# 假设您刚才的失败任务名称是 'rf-tuning-251203-1038'
# 如果rf_tuner对象在内存中，可以不用手动设置
DEFAULT_TUNING_JOB_NAME = 'rf-tuning-251203-1038'

def diagnose_tuning_job_enhanced(tuning_job_name=None):
    """
    增强版诊断函数：自动检查失败原因，并拉取 CloudWatch 详细日志。
    """
    # 1. 区域确定逻辑：优先从 rf_tuner 拿，否则使用默认值
    target_region = 'ap-southeast-1' # 默认猜测是新加坡
    
    if 'rf_tuner' in globals() and hasattr(rf_tuner, 'sagemaker_session'):
        target_region = rf_tuner.sagemaker_session.boto_region_name
    
    print(f"🌍 锁定目标 AWS Region: {target_region}")

    # 2. 任务名称确定逻辑
    tuner_name = None
    if tuning_job_name:
        tuner_name = tuning_job_name
    elif 'rf_tuner' in globals():
        try:
            tuner_name = rf_tuner.latest_tuning_job.name
        except AttributeError:
            pass
    
    if tuner_name is None:
        print(f"❌ 错误: 未传入任务名，内存中也找不到 rf_tuner。请手动指定任务名，例如：'{DEFAULT_TUNING_JOB_NAME}'")
        return

    print(f"🕵️‍♂️ 正在全面诊断任务: {tuner_name}")
    
    # 3. 初始化客户端
    try:
        sm_client = boto3.client('sagemaker', region_name=target_region)
        logs_client = boto3.client('logs', region_name=target_region)
    except Exception as e:
        print(f"❌ AWS 客户端初始化失败: {e}")
        return
    
    log_group_name = '/aws/sagemaker/TrainingJobs'
    
    # 4. 获取子任务列表
    try:
        tuner_resp = sm_client.list_training_jobs_for_hyper_parameter_tuning_job(
            HyperParameterTuningJobName=tuner_name,
            MaxResults=10,
            SortBy='CreationTime',
            SortOrder='Descending'
        )
    except ClientError as e:
        if 'NotFound' in str(e):
            print(f"❌ 在区域 {target_region} 找不到该调优任务 ({tuner_name})。请确认任务名称和区域是否正确。")
        else:
            print(f"❌ AWS API 调用失败: {e}")
        return

    jobs = tuner_resp.get('TrainingJobSummaries', [])
    print(f"🔍 扫描了最近 {len(jobs)} 个子任务...\n")
    
    found_failure = False
    
    for job in jobs:
        job_name = job['TrainingJobName']
        status = job['TrainingJobStatus']
        
        print(f"➡️ 子任务: {job_name} | 状态: {status}")
        
        if status == 'Failed':
            found_failure = True
            print("\n" + "="*70)
            print(f"🚨 锁定首个失败子任务: {job_name}")
            
            # --- A. 获取 Training Job 详细信息（包含 FailureReason） ---
            try:
                job_desc = sm_client.describe_training_job(TrainingJobName=job_name)
                failure_reason = job_desc.get('FailureReason', 'N/A')
                print(f"💥 顶层失败原因 (API Message): {failure_reason}")
                
                # 如果是 OOM 或容器问题，这个原因会很有用
                if 'exit code' in failure_reason or 'InternalServerError' in failure_reason:
                     print("ℹ️ 提示：顶层原因已提供关键信息，请参考其指示。")
            except Exception as e:
                print(f"⚠️ 无法获取 Training Job 描述: {e}")

            # --- B. 获取 CloudWatch 详细日志 ---
            print(f"\n📝 正在尝试拉取 CloudWatch 详细崩溃日志...")
            full_log_messages = []
            
            try:
                streams = logs_client.describe_log_streams(
                    logGroupName=log_group_name,
                    logStreamNamePrefix=job_name
                )
                
                if not streams['logStreams']:
                    print("⚠️ 警告: 该任务未生成日志流，可能启动就失败了。请检查 IAM 权限。")
                else:
                    stream_name = streams['logStreams'][0]['logStreamName']
                    
                    next_token = None
                    while True:
                        # 使用 logs_client.get_log_events 拉取日志
                        # (保持您原有的拉取逻辑，确保能拉取所有事件)
                        kwargs = {
                            'logGroupName': log_group_name,
                            'logStreamName': stream_name,
                            'startFromHead': True,
                            'limit': 1000
                        }
                        if next_token:
                            kwargs['nextToken'] = next_token
                            
                        response = logs_client.get_log_events(**kwargs)
                        
                        for event in response['events']:
                            full_log_messages.append(event['message'].rstrip())
                            
                        next_token = response.get('nextForwardToken')
                        # 当 nextToken 为 None 且 events 数量小于 limit 时，表示日志已拉取完
                        if not next_token or len(response['events']) == 0:
                            break
                    
                    print("\n" + "="*20 + " ☠️ 崩溃现场还原 (关键日志片段) ☠️ " + "="*20)
                    
                    # 智能截取：寻找 Python Traceback 的起点
                    start_index = 0
                    found_traceback = False
                    for i, msg in reversed(list(enumerate(full_log_messages))):
                        # 尝试定位到 Python 错误信息的起始点
                        if 'Traceback (most recent call last)' in msg: 
                            start_index = i
                            found_traceback = True
                            break
                    
                    if found_traceback:
                        print(f"✅ 定位到 Python Traceback 错误，显示后续日志...\n")
                        final_logs = full_log_messages[start_index:]
                    else:
                        # 如果没有 Traceback，就显示最后50行日志
                        print(f"⚠️ 未找到 Traceback 标记，显示最后 50 行日志...\n")
                        final_logs = full_log_messages[-50:]

                    # 打印日志
                    for msg in final_logs:
                        print(msg)
                    print("="*68)
            
            except Exception as e:
                print(f"⚠️ 无法获取日志流或事件: {e}")
                
            print("="*70 + "\n")
            # 找到第一个失败的任务后立即退出，专注于解决它
            break 

    if not found_failure:
        print("\n✅ 当前扫描的子任务中没有发现 'Failed' 状态。")

# --- 运行诊断 ---
# 如果 rf_tuner 对象在内存中，它会尝试自动获取任务名
if 'rf_tuner' in globals():
    try:
        diagnose_tuning_job_enhanced(rf_tuner.latest_tuning_job.name)
    except AttributeError:
        # 如果 rf_tuner 存在但还没有 latest_tuning_job (如流程第一步就失败了)
        print("ℹ️ rf_tuner 对象存在，但无法获取 latest_tuning_job 名称。尝试使用默认任务名...")
        diagnose_tuning_job_enhanced(DEFAULT_TUNING_JOB_NAME)
else:
    # 如果 rf_tuner 对象丢失，使用硬编码的默认任务名
    diagnose_tuning_job_enhanced(DEFAULT_TUNING_JOB_NAME)

🌍 锁定目标 AWS Region: us-east-1
🕵️‍♂️ 正在全面诊断任务: rf-tuning-251203-1038
🔍 扫描了最近 2 个子任务...

➡️ 子任务: rf-tuning-251203-1038-002-2bc3977b | 状态: Failed

🚨 锁定首个失败子任务: rf-tuning-251203-1038-002-2bc3977b
💥 顶层失败原因 (API Message): AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_sklearn_container/training.py", line 31, in train
    entry_point.run(uri=training_environment.module_dir,
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_training/entry_point.py", line 108, in run
    return runner.get(runner_type, user_entry_point, args, env_vars, extra_opts).run(
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_training/process.py", line 424, in run
    p

In [20]:
import time
import sys
import boto3

# ------------------------------------------------------------------
# ⚠️ 注意: 确保在运行这段代码之前，前面的配置代码块已经成功执行，
# 并且 rf_estimator 和 train_input 对象在内存中。
# ------------------------------------------------------------------

# --- 运行 Random Forest Estimator (单次 Fit) ---

# 重新生成一个唯一的 Job Name，确保每次都是新任务
TRAINING_JOB_NAME = f"log-test-rf-{time.strftime('%Y%m%d-%H%M%S')}"

print("\n--- 🚀 Step 5: Launching Single Test Job ---")
print(f"🌲 [1/1] Launching Test Training Job... (Timestamp: {time.strftime('%H:%M:%S')})")
print(f"   -> 任务名: {TRAINING_JOB_NAME}")

try:
    # 使用 rf_estimator 对象，但直接调用 fit，绕过 HyperparameterTuner
    rf_estimator.fit(
        inputs={'train': train_input}, 
        job_name=TRAINING_JOB_NAME, 
        wait=False # 禁用阻塞，我们使用自定义循环
    )
    
except Exception as e:
    print(f"\n❌ 任务启动失败: {e}")
    # 由于我们已经导入了 sys，可以安全地退出
    sys.exit(1)


# --- 状态监控循环 ---
print(f"\n✅ 任务已提交。开始实时监控状态...")
# 确保 boto3 client 使用了正确的 session 和 region
sm_client = rf_estimator.sagemaker_session.sagemaker_client
log_group_name = '/aws/sagemaker/TrainingJobs'
REGION = rf_estimator.sagemaker_session.boto_region_name # 自动从 session 获取区域

while True:
    time.sleep(15) # 每 15 秒查询一次
    try:
        # 获取最新的训练任务状态
        desc = sm_client.describe_training_job(TrainingJobName=TRAINING_JOB_NAME)
        status = desc['TrainingJobStatus']
        
        # 打印当前状态
        secondary_status = desc.get('SecondaryStatus', 'N/A')
        print(f"\r[状态更新] 任务状态: {status:<10} | 容器状态: {secondary_status:<15}", end="", flush=True)
        
        if status in ['Completed', 'Failed', 'Stopped']:
            print(f"\n--- 任务结束，最终状态: {status} ---")
            
            # --- 任务结束后检查 CloudWatch 日志流 ---
            print(f"📝 正在检查 CloudWatch 日志流: {log_group_name}/{TRAINING_JOB_NAME} (区域: {REGION})")
            
            if status == 'Failed':
                failure_reason = desc.get('FailureReason', '无顶层原因')
                print(f"💥 失败原因 (API Message): {failure_reason}")
            
            try:
                logs_client = boto3.client('logs', region_name=REGION)
                streams = logs_client.describe_log_streams(
                    logGroupName=log_group_name,
                    logStreamNamePrefix=TRAINING_JOB_NAME
                )
                
                if streams['logStreams']:
                    print(f"✅ 日志流 FOUND！")
                    print(f"   -> 请立即前往 CloudWatch 控制台，搜索 {TRAINING_JOB_NAME}，查看具体的脚本错误。")
                else:
                    print(f"❌ 日志流 NOT FOUND。问题出在 **IAM 权限** 或 **Git/S3 数据获取**阶段。")

            except Exception as e:
                print(f"❌ 无法查询 CloudWatch 日志流: {e}")

            break
            
    except Exception as e:
        print(f"\n❌ 状态查询失败: {e}")
        break

print("\n🎉 诊断任务执行完毕！")


--- 🚀 Step 5: Launching Single Test Job ---
🌲 [1/1] Launching Test Training Job... (Timestamp: 11:19:14)
   -> 任务名: log-test-rf-20251203-111914


Cloning into '/var/folders/py/wsw9dv2n5g70rl8sm3pxxjg40000gn/T/tmphffkfhxm'...
Already on 'main'


Your branch is up to date with 'origin/main'.


INFO:sagemaker:Creating training-job with name: log-test-rf-20251203-111914



✅ 任务已提交。开始实时监控状态...
[状态更新] 任务状态: Failed     | 容器状态: Failed         
--- 任务结束，最终状态: Failed ---
📝 正在检查 CloudWatch 日志流: /aws/sagemaker/TrainingJobs/log-test-rf-20251203-111914 (区域: us-east-1)
💥 失败原因 (API Message): AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_sklearn_container/training.py", line 31, in train
    entry_point.run(uri=training_environment.module_dir,
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_training/entry_point.py", line 108, in run
    return runner.get(runner_type, user_entry_point, args, env_vars, extra_opts).run(
  File "/miniconda3/lib/python3.9/site-packages/sagemaker_training/process.py", line 424, in run
    proc

/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


❌ 无法查询 CloudWatch 日志流: An error occurred (ResourceNotFoundException) when calling the DescribeLogStreams operation: The specified log group does not exist.

🎉 诊断任务执行完毕！
